In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import kss
import re
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Mecab 객체 생성
mecab = Mecab()

In [3]:
# df 불러오기
df = pd.read_csv("./test.csv", header=None)
df.columns = ['article']

In [4]:
df = df.iloc[0:30]

In [5]:
# 중복 제거
df = df.drop_duplicates('article')
df.reset_index(inplace=True,drop=True)

In [6]:
# kss 모듈(문장분리기)로 문장화
df['sentence'] = [kss.split_sentences(df['article'][i]) for i in range(len(df))]

In [7]:
#각 문장의 길이가 20 이하인건 삭제
for i in range(len(df)):  
    for j in range(len(df['sentence'][i])-1,-1,-1):
        if len(df['sentence'][i][j]) <= 20:
            df['sentence'][i].remove(df['sentence'][i][j])

In [8]:
#문장 전체 길이가 10 이하는 제거(기사를 제거)
df = df[df['sentence'].apply(lambda x: len(x) >= 10)]

In [9]:
df.reset_index(inplace=True,drop=True)

In [10]:
#stopwords list화
temp = pd.read_csv("./stop_words.csv",header = None, usecols = [0], names = ['stopwords'])                                                
stopwords = [temp['stopwords'][i] for i in range(len(temp))]

In [11]:
#Mecab을 활용한 tokenize 및 불용어 제거
result = []
for i in range(len(df)):
    token_list = []
    for sentence in df['sentence'][i]:
        sentence = re.sub('[-=+,#/\?©:^$©@*\"※~&·%｜!』\\‘◇”|\(\)\[\]\<\>`\'’“…》]', '', sentence)
        tokenized = mecab.morphs(sentence)
        for j in range(len(tokenized)-1,-1,-1):
            if tokenized[j] in stopwords:
                tokenized.remove(tokenized[j])
        token_list.append(tokenized)
    result.append(token_list)
df['token'] = result

In [12]:
df

,article,sentence,token
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명..."
1,한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판매...,[한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판...,"[[한국, 관광공사, 중국, 최대, 여행사, 인, 트립, 닷컴, 1, 진행, 한, ..."
2,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ..."
3,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,..."
4,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ..."
5,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3..."
6,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ..."
7,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하..."
8,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ...."
9,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,..."


In [14]:
mecab

In [19]:
# Word2Vec 모델링
# news_tokens = df['token'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [23]:
#벡터화 작업
def vectorize(tokens):
    result = []
    for token in tokens:
        vectors = []
        for sentence in token:
            # 문장 벡터 = 각 단어의 평균 벡터
            init_v = np.array([0.0]*v_dimension)
            count = 0
            for word in sentence:
                word_vectors = model.wv
                if word in word_vectors.vocab:
                    count +=1
                    v = model.wv[word]
                    init_v = init_v + v
            init_v = init_v / count
            vectors.append(init_v)
        result.append(vectors)
    
    frame = {'article' : df['article'], 'sentence': df['sentence'], 'token': df['token'], 'vector': result }
    result = pd.DataFrame(frame) 
    
    return result

In [24]:
df = vectorize(df['token'])

In [25]:
df

,article,sentence,token,vector
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ..."
1,한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판매...,[한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판...,"[[한국, 관광공사, 중국, 최대, 여행사, 인, 트립, 닷컴, 1, 진행, 한, ...","[[-0.08544983807951212, 0.017376393231993113, ..."
2,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0...."
3,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0..."
4,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0...."
5,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83..."
6,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0..."
7,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0...."
8,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0...."
9,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0...."


In [26]:
#코사인유사도 매트릭스
def similarity_matrix(vectors):
    sim_mat = np.zeros([len(vectors), len(vectors)])
    for i in range(len(vectors)):
        for j in range(len(vectors)):
            sim_mat[i][j] = cosine_similarity(vectors[i].reshape(1, v_dimension), vectors[j].reshape(1, v_dimension))[0,0]
    return sim_mat


In [27]:
df['SimMatrix'] = df['vector'].apply(similarity_matrix)

In [28]:
df

,article,sentence,token,vector,SimMatrix
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ...","[[1.0, 0.8390463844682985, 0.38402217976241126..."
1,한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판매...,[한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판...,"[[한국, 관광공사, 중국, 최대, 여행사, 인, 트립, 닷컴, 1, 진행, 한, ...","[[-0.08544983807951212, 0.017376393231993113, ...","[[0.9999999999999994, 0.8753952189311108, 0.22..."
2,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0....","[[1.0, 0.6849624598007038, 0.9491063294782923,..."
3,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0...","[[1.0000000000000004, 0.5431313853998593, 0.29..."
4,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0....","[[0.9999999999999999, 0.020770213584794636, -0..."
5,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83...","[[0.9999999999999997, 0.7537380932719095, 0.85..."
6,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0...","[[1.0000000000000004, 0.5223812823305478, 0.53..."
7,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0....","[[1.0000000000000002, 0.3847769690682543, 0.64..."
8,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0....","[[0.9999999999999997, 0.6869661633995391, 0.68..."
9,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0....","[[0.9999999999999997, 0.34999870105711894, 0.8..."


In [29]:
#코사인 유사도를 활용해서 각 문장마다 점수 매기기
#pagerank 사용
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

In [30]:
df['score'] = df['SimMatrix'].apply(calculate_score)

In [31]:
df

,article,sentence,token,vector,SimMatrix,score
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ...","[[1.0, 0.8390463844682985, 0.38402217976241126...","{0: 0.01795871228825599, 1: 0.0194407872547084..."
1,한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판매...,[한국관광공사와 중국 최대 여행사인 트립닷컴이 1일 함께 진행한 한국 여행 상품 판...,"[[한국, 관광공사, 중국, 최대, 여행사, 인, 트립, 닷컴, 1, 진행, 한, ...","[[-0.08544983807951212, 0.017376393231993113, ...","[[0.9999999999999994, 0.8753952189311108, 0.22...","{0: 0.11038639862752907, 1: 0.1144507171183583..."
2,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0....","[[1.0, 0.6849624598007038, 0.9491063294782923,...","{0: 0.08284668298683674, 1: 0.0785451733839945..."
3,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0...","[[1.0000000000000004, 0.5431313853998593, 0.29...","{0: 0.08138790359620644, 1: 0.0553330965831385..."
4,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0....","[[0.9999999999999999, 0.020770213584794636, -0...","{0: 0.02732887650814984, 1: 0.0510182957113229..."
5,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83...","[[0.9999999999999997, 0.7537380932719095, 0.85...","{0: 0.029298134608698213, 1: 0.031697616647051..."
6,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0...","[[1.0000000000000004, 0.5223812823305478, 0.53...","{0: 0.03955025430747894, 1: 0.0345649053063711..."
7,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0....","[[1.0000000000000002, 0.3847769690682543, 0.64...","{0: 0.060645582370942946, 1: 0.057909049775027..."
8,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0....","[[0.9999999999999997, 0.6869661633995391, 0.68...","{0: 0.06874896041402666, 1: 0.0669202877319058..."
9,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0....","[[0.9999999999999997, 0.34999870105711894, 0.8...","{0: 0.09490203110443626, 1: 0.0565733248059762..."


# Result - base

In [59]:
def ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s, i) for i,s in enumerate(sentences)), 
                        key=lambda x : x[0], reverse=True)
    top_n_sentences = [sentence for score, sentence, order in top_scores[:n]]
    return " ".join(top_n_sentences)

In [60]:
df['summary'] = df.apply(lambda x: 
                            ranked_sentences(x.sentence, 
                            x.score), axis=1)

In [61]:
temp = df[['article','summary']]

In [62]:
for i in range(3,5):
    print(i+1,'번 문서')
    print('원문 :',temp.loc[i].article)
    print('')
    print('요약 :',temp.loc[i].summary)
    print('')


4 번 문서
원문 : 롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있어…1인당 구매 제한 없어 앵커 품절 대란으로 쉽게 못 구했던 침방울 차단 마스크가 대형 마트와 백화점 편의점 등 전국 오프라인 매장에 본격적으로 풀렸습니다. 아직 초반이라 물량이 많지 않지만 점차 더 많이 공급되면서 수급이 원활해질 수 있을지 주목됩니다. 차유정 기자입니다. 기자 마트 개점 30분 전부터 긴 줄이 생겼습니다. 침방울 차단용 마스크 사려는 사람들입니다. 해당 지점에서 준비한 물량 천6백 장은 30분 만에 동났습니다. 허희정 서울 응암동 여름이니까 너무 더우니까 비말이 시원하다고 해서 사러 왔습니다. 언제 오셨어요 9시 15분쯤에요. 무더운 여름철에 숨쉬기 편해 인기가 많은 침방울 차단 마스크가 오프라인 매장에 본격적으로 풀렸습니다. 이마트에 이어 롯데마트와 홈플러스도 침방울 차단 마스크 판매를 본격화한 겁니다. 롯데마트는 장당 5백 원씩 전 점포에서 일단 16만 장을 팔고 홈플러스도 장당 5 6백 원에 이번 주에만 30만 장을 판매할 계획입니다. 대부분 편의점에서도 살 수 있게 됐습니다. 특히 편의점은 1인당 구매 제한이 없습니다. 지난달부터 판매를 개시한 GS25 이외에 세븐일레븐은 전 점포에서 하루 평균 5만 장을 CU는 전국 만4천 개 점포에서 한 주에 50만 장을 이마트 24는 첫 주에만 백만 장을 공급합니다. 장당 가격은 CU와 세븐일레븐이 각각 6백 원 이마트 6백 원과 975원 등으로 대부분 5개 묶음 단위로 판매됩니다. 아직은 한 지점에는 팔 수 있는 물량이 충분치 않지만 물량 공급을 점차 늘려나갈 계획입니다. 백화점도 판매에 나서 롯데는 장당 760원에 신세계는 장당 580 700원에 공급을 시작했습니다. 침방울 차단용 마스크가 전국 오프라인 매장에 본격적으로 풀리기 시작하면서 수급이 원활해질지 수 있을지 주목됩니다. YTN 차유정입니다.

요약 : 롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 

# Result - seq

In [63]:
# score가 높은 순으로 3개만 문장 뽑기
def ranked_sentences_seq(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s, i) for i,s in enumerate(sentences)), 
                        key=lambda x : x[0], reverse=True)
    top_n_sentences_seq = sorted([(order,sentence) for score, sentence, order in top_scores[:n]]
                             , key=lambda x:x[0])
    top_n_sentences = list(map(lambda x:x[1], top_n_sentences_seq))
    return " ".join(top_n_sentences)

In [64]:
df['summary'] = df.apply(lambda x: 
                            ranked_sentences_seq(x.sentence, 
                            x.score), axis=1)

In [65]:
temp = df[['article','summary']]

In [66]:
for i in range(3,5):
    print(i+1,'번 문서')
    print('원문 :',temp.loc[i].article)
    print('')
    print('요약 :',temp.loc[i].summary)
    print('')


4 번 문서
원문 : 롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있어…1인당 구매 제한 없어 앵커 품절 대란으로 쉽게 못 구했던 침방울 차단 마스크가 대형 마트와 백화점 편의점 등 전국 오프라인 매장에 본격적으로 풀렸습니다. 아직 초반이라 물량이 많지 않지만 점차 더 많이 공급되면서 수급이 원활해질 수 있을지 주목됩니다. 차유정 기자입니다. 기자 마트 개점 30분 전부터 긴 줄이 생겼습니다. 침방울 차단용 마스크 사려는 사람들입니다. 해당 지점에서 준비한 물량 천6백 장은 30분 만에 동났습니다. 허희정 서울 응암동 여름이니까 너무 더우니까 비말이 시원하다고 해서 사러 왔습니다. 언제 오셨어요 9시 15분쯤에요. 무더운 여름철에 숨쉬기 편해 인기가 많은 침방울 차단 마스크가 오프라인 매장에 본격적으로 풀렸습니다. 이마트에 이어 롯데마트와 홈플러스도 침방울 차단 마스크 판매를 본격화한 겁니다. 롯데마트는 장당 5백 원씩 전 점포에서 일단 16만 장을 팔고 홈플러스도 장당 5 6백 원에 이번 주에만 30만 장을 판매할 계획입니다. 대부분 편의점에서도 살 수 있게 됐습니다. 특히 편의점은 1인당 구매 제한이 없습니다. 지난달부터 판매를 개시한 GS25 이외에 세븐일레븐은 전 점포에서 하루 평균 5만 장을 CU는 전국 만4천 개 점포에서 한 주에 50만 장을 이마트 24는 첫 주에만 백만 장을 공급합니다. 장당 가격은 CU와 세븐일레븐이 각각 6백 원 이마트 6백 원과 975원 등으로 대부분 5개 묶음 단위로 판매됩니다. 아직은 한 지점에는 팔 수 있는 물량이 충분치 않지만 물량 공급을 점차 늘려나갈 계획입니다. 백화점도 판매에 나서 롯데는 장당 760원에 신세계는 장당 580 700원에 공급을 시작했습니다. 침방울 차단용 마스크가 전국 오프라인 매장에 본격적으로 풀리기 시작하면서 수급이 원활해질지 수 있을지 주목됩니다. YTN 차유정입니다.

요약 : 롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 